In [1]:
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
from torch import nn
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from tqdm import tqdm
# from sklearn.metrics import accuracy_score

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
train_data = FashionMNIST("./dataset", train=True, transform=ToTensor(), download=True)
test_data = FashionMNIST("./dataset", train=False, transform=ToTensor(), download=True)
# See first training sample

image, label = train_data[0]
image, label


(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.0000, 0.0000, 0.0510,
           0.2863, 0.0000, 0.0000, 0.0039, 

In [2]:
# What's the shape of the image?
image.shape

torch.Size([1, 28, 28])

In [3]:
# See classes
class_names = train_data.classes
class_names


['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [4]:
BATCH_SIZE = 32

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)
len(train_dataloader)


1875

In [5]:
batch_x, batch_y = next(iter(train_dataloader))
batch_x.shape, batch_y.shape


(torch.Size([32, 1, 28, 28]), torch.Size([32]))

In [6]:
ss = nn.Flatten()
ss(batch_x).shape

torch.Size([32, 784])

In [7]:

test_layer = nn.Sequential(
                nn.Conv2d(1, BATCH_SIZE, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(BATCH_SIZE),
                nn.ReLU(),
            )
test_layer(batch_x).shape

torch.Size([32, 32, 28, 28])

In [8]:
torch.cuda.manual_seed(42)
class FassionMNISTCNN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__()
        self.cnn_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=1,
                      out_channels=10,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=10,
                     out_channels=10,
                     kernel_size=3,
                     padding=1,
                     stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        #block 2
        self.cnn_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=10,
                     out_channels=10,
                     kernel_size=3,
                     padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=10,
                     out_channels=10,
                     kernel_size=3,
                     padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        #out layer classifier
        self.class_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=10*7*7,
                     out_features=10)
        )
        
    def forward(self, x):
        x = self.cnn_block_1(x)
#         print("cnn block 1", x.shape)
        x = self.cnn_block_2(x)
#         print("cnn block 2", x.shape)
        x = self.class_layer(x)
#         print("cnn block 3", x.shape)
        return x


In [ ]:
FasNetCnn = FassionMNISTCNN().to(device=device)
next(FasNetCnn.parameters()).device

device(type='cuda', index=0)

In [10]:
batch_x = batch_x.to(device)
FasNetCnn(batch_x).shape

torch.Size([32, 10])

In [11]:
loss_fn = nn.CrossEntropyLoss().to(device=device)
optim_fn = torch.optim.Adam(FasNetCnn.parameters(), lr=0.008)

In [12]:
total_params = 0
for p in FasNetCnn.parameters():
    total_params+=p.numel()
print(total_params)

7740


In [13]:
def acc_fn(y_preds, y):
    acc_score = torch.eq(y_preds, y).sum() / len(y)
    return acc_score

In [14]:
epochs = 10
for e in range(epochs):
    total_train_loss = 0
    total_train_acc = 0

    for batch_x_train, batch_y_train in tqdm(train_dataloader):
        FasNetCnn.train()
        batch_x_train = batch_x_train.to(device)
        batch_y_train = batch_y_train.to(device)

        y_logits = FasNetCnn(batch_x_train)
        train_loss = loss_fn(y_logits, batch_y_train)
        total_train_loss+=train_loss

        total_train_acc+=acc_fn(torch.argmax(torch.softmax(y_logits, dim=1),dim=1), batch_y_train)

        optim_fn.zero_grad()
        train_loss.backward()
        optim_fn.step()
    
    with torch.inference_mode():
        total_train_loss = total_train_loss / len(train_dataloader)
        total_train_acc = total_train_acc / len(train_dataloader)
        print("Total Train loss: ",total_train_loss, "Train Acc: ",total_train_acc)




100%|██████████| 1875/1875 [00:07<00:00, 248.52it/s]


Total Train loss:  tensor(0.4558, device='cuda:0') Train Acc:  tensor(0.8321, device='cuda:0')


100%|██████████| 1875/1875 [00:07<00:00, 260.00it/s]


Total Train loss:  tensor(0.3490, device='cuda:0') Train Acc:  tensor(0.8742, device='cuda:0')


100%|██████████| 1875/1875 [00:07<00:00, 265.69it/s]


...

Total Train loss:  tensor(0.3122, device='cuda:0') Train Acc:  tensor(0.8862, device='cuda:0')


100%|██████████| 1875/1875 [00:07<00:00, 257.80it/s]


Total Train loss:  tensor(0.3172, device='cuda:0') Train Acc:  tensor(0.8836, device='cuda:0')


100%|██████████| 1875/1875 [00:07<00:00, 259.10it/s]

Total Train loss:  tensor(0.3075, device='cuda:0') Train Acc:  tensor(0.8873, device='cuda:0')


In [17]:
torch.save(FasNetCnn.state_dict(), "fasMODcnn_state.pt")

In [15]:
# del FasNetCnn
# import gc         # garbage collect library
# gc.collect()
# torch.cuda.empty_cache()
import torch
FasNetCnn.load_state_dict(torch.load("fasMODcnn_state.pt"))

/tmp/ipykernel_113803/4057175094.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  FasNetCnn.load_state_dict(torch.load("fasMODcnn_state.pt"))


<All keys matched successfully>

In [15]:

total_test_loss, total_test_acc = 0,0
# acc on test data
for idx, (batch_test_x, batch_test_y) in enumerate(tqdm(test_dataloader)):
    with torch.inference_mode():
        batch_test_x = batch_test_x.to(device)
        batch_test_y = batch_test_y.to(device)

        
        y_logits = FasNetCnn(batch_test_x)
        test_loss = loss_fn(y_logits, batch_test_y)
        total_test_loss+=test_loss

        total_test_acc+=acc_fn(torch.argmax(torch.softmax(y_logits, dim=1), dim=1),batch_test_y)

    # print("Train loss:", train_loss)
with torch.inference_mode():
    total_test_loss = total_test_loss / len(test_dataloader)
    total_test_acc = total_test_acc / len(test_dataloader)
    print("Total Test loss: ",total_test_loss, "Test Acc: ",total_test_acc)

100%|██████████| 313/313 [00:00<00:00, 342.00it/s]

Total Test loss:  tensor(0.3381, device='cuda:0') Test Acc:  tensor(0.8786, device='cuda:0')
